In [1]:
# imports
from collections import Counter
from itertools import islice
import re
import pandas as pd
import string
import csv
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
import math
porter = PorterStemmer()
# #########################
# DATA CLEANING TECHNIQUES
# #########################
# split by sentance
# remove puncuation
# lowercase letters
# remove/convert numbers
# -------------------------

# #########################
# NLP TECHNIQUES
# #########################
# stemming
# -------------------------

In [2]:
stopwords = {"I", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "between", "into", "through", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "that", "thats", "that's", "than", "too", "very", "can", "will", "just", "should", "now"}
# sent1 = "i am sam"
# sent2 = "sam i am"
# sent3 = "i do not like green eggs and ham"
# sent4 = "the quick person did not realize his speed and the quick person bumped "


# # words = re.findall("\w+", sent4)
# # bigramCount = dict(Counter(zip(words, islice(words, 1, None))))

# # for i in bigramCount.keys():
# #     print(str(i) + " count: %d" % bigramCount[i])

# load in the crawled data and perform some cleaning
news = pd.read_csv(r"C:\Users\Rogith\Desktop\Notebooks\newsgrams\newsutf8.csv")
print(news.shape[0])
news = news[:200]
for i in range(news.shape[0]):
    s = news.iloc[i][4][2:]
    news.at[i,'body'] = s
# print(news.iloc[1][4])
print("done")

7613
done


In [3]:
gibberish = ['\(CNN\)', 'CNN', 'Getty Images', '\\\\xc3', '\\\\x94', '\\\\x9c', '\\\\x9d', '\\\\x95', '\\\\xb3', '\\\\xe2', '\\\\x80', '\\\\x99', '\\\\xa9', '\\\\xf0', '\\\\x9f', '\\\\x91']
for i in range(len(gibberish)):
    for j in range(news.shape[0]):
        output = news.iloc[j][4]
        output = re.sub(gibberish[i], '', output)
        output = re.sub('\\\\n', ' ', output)
        output = re.sub('\\\\\\\'', '', output)
        output = re.sub('\'', '', output)
        news.at[j,'body'] = output

print("#################################################")
# for i in range(news.shape[0]):
#     print(news.at[i, 'body'])
#     print("#################################################")
print(news.iloc[0][4])

#################################################
 Now thats one helluva Costco run.  A grocer on a remote Alaskan island thats only accessible by boat or plane is going above and beyond to keep his small city fed during the coronavirus pandemic.  Toshua Parker, owner of Icy Strait Wholesale in Gustavus, has been making a weekly boat trip with his staff to Juneau, the states capital 50 miles away, to restock on essential food and supplies at Costco. It takes them about 14 hours to complete the journey on a 96 feet long converted military landing craft.  Gustavus is a coastal community that borders Glacier Bay National Park. For the citys 450 residents, the only place to buy groceries is ToshCo, the locals nickname for Parkers store.  Parker usually has food and supplies shipped from Costco to his store aboard the states ferry system, but its no longer running to Gustavus because of the pandemic, as well as damage caused to the citys dock by severe storms.  Read More"


In [4]:
collection = str.maketrans('', '', string.punctuation)
dterms = dict()
for j in range(news.shape[0]):    
    tokens = word_tokenize(news.iloc[j][4])
    tokens = [t.lower() for t in tokens]
    depunc = [t.translate(collection) for t in tokens]
    words = [w for w in depunc if not w  == '']
    cleaner = [w for w in words if not w in stopwords]
    stemmed = [porter.stem(word) for word in cleaner]
    dterms[j] = stemmed
#     dterms[j] = cleaner


In [5]:
term_list = []
doc_index = {}
term_index = {}



#function that creates termlist and doc_idx as we count each doc_length
def make_term_list_and_doc_index():
    for doc in range(len(dterms)):
        count = 0
        for word in dterms[doc]:
            count += 1
            term_list.append((word, doc))
        doc_index[doc] = count

def update_term(item):
    i = 0
    for listitem in term_index[item[0]]:
        #if word_docName == encounterd_fileID
        if (item[1] == listitem[0]):
            listitem[1] += 1
            return
        else:
            i += 1
    term_index[item[0]].append( [item[1] , 1 ] )
    return


def make_term_index():
    for item in term_list:
        if item[0] not in term_index:
            # add term
            term_index[item[0]] = [[item[1], 1]]
        else:
            # update term
            update_term(item)
            
make_term_list_and_doc_index()
make_term_index()
# i = 0
# print(len(term_index))
# for key, value in term_index.items(): 
#     print(key, ":", value)
#     if i > 40:
#         break

In [6]:
def get_term_data(term):
    print(term + ' -> ')
    if term in term_index:
        i = 0
        for postings in term_index[term]:
            tf = postings[1]/doc_index[postings[0]] #num_occur/doc_size
            idf = math.log(len(doc_index)/len(term_index[term]), 2)
            tf_idf = tf*idf
            print('Posting: Doc# ' + str(postings[0]) + ', tf: ' + str(tf) + ', idf: ' + str(idf) + ', tf-idf: ' + str(tf_idf))
            if i > 14:
                break
            i += 1
    else:
        print('Term not found')
        
# i = 0
# for key, value in term_index.items(): 
#     print(key)
#     print(len(term_index[key]))
#     if i > 34:
#         break
#     i += 1


In [ ]:
# initialize the set and the dataframe with the first word of the first document
# words_encountered = set(dterms[0][0])
# unigrams = pd.DataFrame({dterms[0][0]: [0]})

# for i in range(news.shape[0]):
#     if i > 0:
#         tdf = pd.DataFrame([[0]*len(unigrams.columns)], columns=unigrams.columns)
#         t = unigrams.append(tdf,ignore_index=True)
#         unigrams = t
#     for word in dterms[i]:
#         if word not in words_encountered:
#             words_encountered.add(word)
#             col_index = len(unigrams.columns)
#             temp = pd.DataFrame({word: [0]*(i+1)})
#             t2 = unigrams.join(temp)
#             unigrams = t2
#             unigrams.at[i, word] = 1
#         else:
#             cols = list(unigrams.columns)
#             unigrams.at[i, word] += 1
# too slow with dataframes ^ 

# for i in range(10):
#     print(dterms[i][:6])

# unigrams.to_csv('unigrams2.csv', encoding='utf-8')


In [7]:
# for k in dterms.keys():
#     print(k, dterms[k][0])

a_file = open("TermDocMatrix.csv", "w")

writer = csv.writer(a_file)
for key, value in term_index.items():
    writer.writerow([key, value])

a_file.close()
